In [1]:
from platypus import NSGAII, Problem, Real
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
from Mult_Obj_Helathy import Main_Model as Mdl_H 
from Mult_Obj_Tumor import Main_Model as Mdl_T

# Mixture of CAR NK and WT
(1) check for 50 and 50 

In [4]:
Shell_H = Mdl_H()
Shell_T = Mdl_T()

In [6]:
Tumor_No = 10000
for i in [10.0,5.0,2.5,1.25,0.625,0.3125]:
    x0 = i
    NK_No = x0*Tumor_No
    y0 = 1.0
    x = [0.9791789663709559,0.8936323663149114]
    frac = 0.8
    Shell_T.Tumor(x[0], frac, x[1], NK_No=NK_No, Tc_No = Tumor_No)
    Shell_H.Healthy(x[0], frac, x[1], NK_No=NK_No, Tc_No = Tumor_No)
    print('done')

T 53.392371004926865
H 75.60295376823717
done
T 41.94746013722439
H 62.40345581382408
done
T 31.099191650434076
H 44.96771135546004
done
T 20.869497872618325
H 28.209560514716458
done
T 12.508050858428177
H 16.00039385689085
done
T 6.91774723086882
H 8.551013356434856
done


# Tumor Cell
### (i) CAR_killing
    1/78.94391239956171 == 0.012667221190389747
### (ii) WT killing
    1/40.27411983691734 ==0.02482984119949279

# Healthy Cell
### (i) CAR_killing
    33.209108322965996
### (ii) WT killing
    7.810341768315665

In [5]:
# Define your multi-objective problem here
class MyProblem(Problem):
    def __init__(self,ET_ratio):
        super(MyProblem, self).__init__(3, 2) # #variables, #objectives
        self.types[:] = [Real(0.0, 1.0),Real(0.001, 1.0),Real(0.0, 1.0)]
        self.Shell_H = Mdl_H()
        self.Shell_T = Mdl_T()
        self.ET_ratio = ET_ratio
        self.directions[:] = [Problem.MINIMIZE, Problem.MAXIMIZE]

    def evaluate(self, solution):
        x = np.array(solution.variables)
        print('x0',x)
        Tumor_No = 10000
        NK_No = self.ET_ratio*Tumor_No
        f1 = self.Shell_H.Healthy(x[0], x[1], x[2], NK_No=NK_No, Tc_No = Tumor_No) # Objective 1
        f2 = self.Shell_T.Tumor(x[0], x[1], x[2], NK_No=NK_No, Tc_No = Tumor_No) # Objective 2 
        #f1 = self.Shell_H.Healthy(self.frac,x[0],x[1], NK_No=NK_No, Tc_No = Tumor_No)   # Objective 2
        #f2 = self.Shell_T.Tumor(self.frac,x[0],x[1], NK_No=NK_No, Tc_No = Tumor_No)  # Objective 1
        solution.objectives[:] = [f1, f2]
    

In [6]:
for ET_ratio in [10.0]:
        problem = MyProblem(ET_ratio)
        algorithm = NSGAII(problem)
        algorithm.run(2)
        Healthy_Lysis = [s.objectives[0] for s in algorithm.result]
        Tumor_Lysis = [s.objectives[1] for s in algorithm.result]
        alpha1 =  [s.variables[0] for s in algorithm.result]
        alpha4 =  [s.variables[1] for s in algorithm.result]
        frac =  [s.variables[2] for s in algorithm.result]
        filename = f'a4_bw_p001_1_Lysis_Kasumi_Healthy_CAR_NK_ET_{ET_ratio}.xlsx'
        df = pd.DataFrame(columns=['alpha1','alpha4','w','Tumor_lysis', 'Healthy_Lysis'])
        df['alpha1'] = alpha1
        df['alpha4'] = alpha4
        df['w'] = frac
        df['Tumor_lysis'] = Tumor_Lysis
        df['Healthy_Lysis'] = Healthy_Lysis
        df.to_excel(filename)
        print('Done for', ET_ratio)

x0 [0.94030484 0.75798966 0.76514824]
H 84.63224036740438
T 64.47587218790731
x0 [0.55131053 0.34608956 0.50066513]
H 91.78546713320782
T 84.29670863298695
x0 [0.91306928 0.27119383 0.22215024]
H 99.99652352727338
T 99.70233483414592
x0 [0.20120385 0.99778096 0.44129018]
H 93.9508865268493
T 88.3029309929651
x0 [0.07848602 0.44971239 0.68467085]
H 45.84225084702195
T 56.94764574541114
x0 [0.13179275 0.6589751  0.0847137 ]
H 100.00000711661036
T 99.99978959336083
x0 [0.94148964 0.39496139 0.74060404]
H 79.8633945332325
T 61.515436428459644
x0 [0.08077116 0.5708185  0.74662183]
H 38.35932463156622
T 51.20384211754949
x0 [0.40370408 0.16461103 0.73571374]
H 46.8907285303071
T 53.70212784355832
x0 [0.39531989 0.04503285 0.37986316]
H 94.26773000655267
T 91.18211750488256
x0 [0.88859204 0.62143747 0.08004351]
H 100.00001185867038
T 99.99988910998096
x0 [0.75468767 0.70828909 0.34279732]
H 99.7461799751857
T 97.38429571736802
x0 [0.42116189 0.54386781 0.14186689]
H 99.99991861669233
T 99.986